# TDDA: Test-Driven Data Analysis

[TDDA](https://github.com/tdda/tdda) verwendet Dateieingaben (wie NumPy-Arrays oder Pandas DataFrames) und eine Reihe von Einschränkungen (engl.: _constraints_), die als JSON-Datei gespeichert werden.

* `tdda.referencetest` unterstützt die Erstellung von Referenztests, die entweder auf `unittest` oder `pytest` basieren.
* `tdda.constraints` wird verwendet, um Constraints aus einem (Pandas)-DataFrame zu ermitteln, sie als JSON auszuschreiben und zu überprüfen, ob Datensätze die Constraints in der Constraints-Datei erfüllen. Es unterstützt auch Tabellen in einer Vielzahl von relationalen Datenbanken.
* `tdda.rexpy` ist ein Werkzeug zur automatischen Ableitung von regulären Ausdrücken aus einer Spalte in einem Pandas DataFrame oder aus einer (Python)-Liste von Beispielen.

## 1. Importe

In [1]:
import pandas as pd
import numpy as np
from tdda.constraints import discover_df, verify_df

In [2]:
df = pd.read_csv('iot_example.csv')

## 2. Daten überprüfen

Mit [pandas.DataFrame.sample](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html) lassen wir uns die ersten zehn Datensätze anzeigen:

In [3]:
df.sample(10)

,timestamp,username,temperature,heartrate,build,latest,note
39897,2017-01-17T10:34:58,starknicholas,9,63,8aa3d627-c9b4-b57e-bb32-012b7ad30033,1,sleep
56184,2017-01-23T22:48:32,hendersonsteven,11,70,1feb8055-fbe9-88dc-ddea-01d35aadc6f3,0,user
79080,2017-02-02T03:07:30,wardtimothy,7,79,787bcdfb-4a56-d377-a6ee-c534ad477814,1,NaN
59493,2017-01-25T06:39:51,xevans,27,74,6aa43c1d-2e74-3247-fa8d-699e38a4a0bd,0,interval
143746,2017-02-27T22:40:25,aaron53,18,83,c61a9b30-404c-d76a-19a4-22e27b6727c7,0,user
39847,2017-01-17T10:05:42,amynichols,20,64,ac97a2bb-f7db-976c-4063-836c3a931345,0,user
20457,2017-01-09T16:02:10,jonessarah,22,84,9a5e10ab-477c-793d-312e-957ff63031e5,0,NaN
123208,2017-02-19T17:19:59,jperkins,15,70,2eb83fa8-b99c-9e70-a89e-a0003aef7c57,0,sleep
80161,2017-02-02T13:27:04,allenjones,6,72,ec921130-a5e3-9b0a-e9aa-88cee0e74b7c,1,user
145244,2017-02-28T13:00:35,davidreese,19,72,a4f85ee7-7a79-a400-71f9-6f2316b55ebb,0,wake


Und mit [pandas.DataFrame.dtypes](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html) lassen wir uns die Datentypen für die einzelnen Spalten anzeigen:

In [4]:
df.dtypes

timestamp      object
username       object
temperature     int64
heartrate       int64
build          object
latest          int64
note           object
dtype: object

## 3. Erstellen eines _constraints_-Objekt

Mit `discover_constraints` kann ein Vonstraints-Objekt erzeugt werden.

In [5]:
constraints = discover_df(df)

In [6]:
constraints

In [7]:
constraints.fields

Fields([('timestamp',
         <tdda.constraints.base.FieldConstraints at 0x7fd9c43e4b50>),
        ('username',
         <tdda.constraints.base.FieldConstraints at 0x7fd9c1495a90>),
        ('temperature',
         <tdda.constraints.base.FieldConstraints at 0x7fd9c1495df0>),
        ('heartrate',
         <tdda.constraints.base.FieldConstraints at 0x7fd9c14a8190>),
        ('build', <tdda.constraints.base.FieldConstraints at 0x7fd9c14a84f0>),
        ('latest', <tdda.constraints.base.FieldConstraints at 0x7fd9c14a8850>),
        ('note', <tdda.constraints.base.FieldConstraints at 0x7fd9c14a8b80>)])

## 4. Schreiben der _Constraints_ in eine Datei

In [8]:
with open('../../data/ignore-iot_constraints.tdda', 'w') as f:
    f.write(constraints.to_json())

Wenn wir uns die Datei genauer betrachten können wir erkennen, dass z.B. für die `timestamp`-Spalte eine Zeichenkette mit 19 Zeichen erwartet wird und `temperature` Integer mit Werten von 5–29 erwartet.

In [9]:
cat ../../data/ignore-iot_constraints.tdda

{
    "creation_metadata": {
        "local_time": "2020-07-06 14:14:58",
        "utc_time": "2020-07-06 12:12:58",
        "creator": "TDDA 1.0.31",
        "host": "eve.local",
        "user": "veit",
        "n_records": 146397,
        "n_selected": 146397
    },
    "fields": {
        "timestamp": {
            "type": "string",
            "min_length": 19,
            "max_length": 19,
            "max_nulls": 0,
            "no_duplicates": true
        },
        "username": {
            "type": "string",
            "min_length": 3,
            "max_length": 21,
            "max_nulls": 0
        },
        "temperature": {
            "type": "int",
            "min": 5,
            "max": 29,
            "sign": "positive",
            "max_nulls": 0
        },
        "heartrate": {
            "type": "int",
            "min": 60,
            "max": 89,
            "sign": "positive",
            "max_nulls": 0
        },
        "build": {
            "type": "string"

## 5. Überprüfen von Dataframes mit `verify_df`

In [12]:
new_df = pd.read_csv('iot_example_with_nulls.csv')
new_df.sample(10)

,timestamp,username,temperature,heartrate,build,latest,note
8818,2017-01-05T00:34:21,harmstrong,9.0,69,705516c5-fd62-feed-6132-b6f6b71a4432,NaN,wake
138815,2017-02-25T23:17:20,bcarter,22.0,81,7e8b8857-f8fa-9706-4ca2-b34df07a0f71,0.0,wake
85685,2017-02-04T18:30:05,jamesbowers,24.0,64,9ef319f5-f9a5-2b57-a9a6-cdba73ff5e0a,0.0,sleep
111712,2017-02-15T03:28:26,twilson,9.0,87,c7312d98-96c0-5c91-1454-bf9877f59c97,0.0,user
137874,2017-02-25T14:13:33,lortiz,6.0,79,c0dc0221-4cc0-ee70-6b41-7b5f599f2c18,NaN,user
80105,2017-02-02T12:55:29,spencekimberly,NaN,68,4153f98f-a392-201d-de11-08b1b3fade2c,NaN,NaN
67483,2017-01-28T11:23:34,higginsderek,27.0,74,03b16390-1ca5-460f-8274-f50fe226b0ee,0.0,user
73082,2017-01-30T17:20:00,kevinsimmons,19.0,75,1dd63d93-d4ef-2618-55ee-9883899d4c21,1.0,sleep
41612,2017-01-18T03:11:24,jprice,18.0,85,bee3e5be-9ee5-7f31-b888-6d569bcc14b2,1.0,NaN
113126,2017-02-15T17:04:13,lisaunderwood,NaN,85,7ab4a7e3-9635-52b0-3c98-0ec35aebedaa,0.0,NaN


In [11]:
v = verify_df(new_df, '../../data/ignore-iot_constraints.tdda')